### Estimate the superconducting penetration depth of $\beta$-Ta

In [ ]:
from pathlib import Path
import corner
import lmfit
import numpy as np
from scipy.constants import mu_0
from uncertainties import ufloat

from betata import plt, get_blues, get_purples
import matplotlib.ticker as ticker
from betata.resonator_studies.resonator import (
    Resonator,
    load_resonators,
    save_resonator,
)

# assign a more reasonable-ish uncertainty for N_sq
SIM_UNCERTAINTY = 0.05
# assign a reasonable-ish uncertainty for film thickness
THICKNESS_UNCERTAINTY = 0.05

CWD = Path.cwd()

Define helper functions to calculate and save sheet inductances for all resonators

In [ ]:
def find_sheet_inductance(l_kin, n_sq) -> ufloat:
    """ """
    return l_kin / n_sq

def save_resonator_sheet_inductance():
    resonators: list[Resonator] = load_resonators()
    for resonator in resonators:
        if None not in [resonator.l_kin, resonator.l_kin_err, resonator.N_sq, resonator.N_sq_err]:
            u_l_kin = ufloat(resonator.l_kin, resonator.l_kin_err)
            u_n_sq = ufloat(resonator.N_sq, SIM_UNCERTAINTY * resonator.N_sq + resonator.N_sq_err)

            u_l_sheet = find_sheet_inductance(u_l_kin, u_n_sq)

            resonator.l_sheet = u_l_sheet.n
            resonator.l_sheet_err = u_l_sheet.s

            save_resonator(resonator)

In [ ]:
save_resonator_sheet_inductance()

Load resonators and extract thickness and sheet inductance data

In [ ]:
resonators = load_resonators()

In [ ]:
# only use CPWs for this subfigure
thicknesses = []
l_sheets = []
l_sheet_errs = []
for resonator in resonators:
    l_kin, n_sq, n_sq_err = resonator.l_kin, resonator.N_sq, resonator.N_sq_err
    if resonator.type == "CPW" and None not in [l_kin, n_sq, n_sq_err]:
        thicknesses.append(resonator.film_thickness)
        l_sheets.append(resonator.l_sheet)
        l_sheet_errs.append(resonator.l_sheet_err)
thicknesses = np.array(thicknesses)
sort_idx = np.argsort(thicknesses)
thicknesses = thicknesses[sort_idx]
thickness_errs = thicknesses * THICKNESS_UNCERTAINTY
l_sheets = np.array(l_sheets)[sort_idx]
l_sheet_errs = np.array(l_sheet_errs)[sort_idx]

Do initial fit

In [ ]:
def pen_depth_fit_fn(x, pen_depth):
    """ """
    return (mu_0 * pen_depth) / np.tanh(x / pen_depth)

In [ ]:
fit_result = lmfit.Model(pen_depth_fit_fn).fit(
    l_sheets,
    x=thicknesses,
    weights=l_sheet_errs,
    pen_depth=1.63e-6,
)
print(fit_result.fit_report())

Run MCMC to refine fit result

In [ ]:
def pen_depth_error_fn(params, x, data, l_sheet_errs):
    """ """
    pen_depth = params["pen_depth"].value
    return (data - pen_depth_fit_fn(x, pen_depth)) / l_sheet_errs


In [ ]:
emcee_kws = {
    "nwalkers": 250,
    "burn": 1000,
    "steps": 10000,
    "thin": 20,
    "is_weighted": False,
}

emcee_params = fit_result.params
emcee_params.add('__lnsigma', value=np.log(0.1), min=-np.inf, max=np.inf)
emcee_params["pen_depth"].min = 1e-7
emcee_params["pen_depth"].max = 1e-5


emcee_result = lmfit.minimize(
    pen_depth_error_fn,
    params=emcee_params,
    args=(thicknesses, l_sheets, l_sheet_errs),
    method="emcee",
    nan_policy="omit",
    **emcee_kws,
)

Make corner plot

In [ ]:
corner_fig = plt.figure(figsize=(8, 8))
emcee_plot = corner.corner(
    emcee_result.flatchain,
    labels=emcee_result.var_names,
    truths=list(emcee_result.params.valuesdict().values()),
    fig=corner_fig,
)
emcee_plot.tight_layout()

acc_frac_fig, acc_frac_ax = plt.subplots(figsize=(6, 4))
acc_frac_ax.plot(emcee_result.acceptance_fraction, 'o')
acc_frac_ax.set_xlabel('Walker')
acc_frac_ax.set_ylabel('Acceptance fraction')
if hasattr(emcee_result, "acor"):
    print("Autocorrelation time for the parameters:")
    print("----------------------------------------")
    for i, p in enumerate(emcee_params):
        print(f'{p} = {emcee_result.acor[i]:.3f}')

print("Median of posterior probability distribution")
print("--------------------------------------------")
print(lmfit.fit_report(emcee_result.params))

highest_prob = np.argmax(emcee_result.lnprob)
hp_loc = np.unravel_index(highest_prob, emcee_result.lnprob.shape)
mle_soln = emcee_result.chain[hp_loc]
for i, par in enumerate(emcee_params):
    emcee_params[par].value = mle_soln[i]

In [ ]:
final_pen_depth = ufloat(emcee_result.params["pen_depth"].value, emcee_result.params["pen_depth"].stderr)
print(f"Superconducting penetration depth λ: {final_pen_depth * 1e6:.2f} μm")

Plot data and fit

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

thicknesses_um = thicknesses * 1e6
thickness_errs_um = thickness_errs * 1e6
l_sheets_pH = l_sheets * 1e12
l_sheet_errs_pH = l_sheet_errs * 1e12

thicknesses_dummy_1 = np.linspace(20e-9, 2e-6, 101)
l_sheets_fit_pH_1 = pen_depth_fit_fn(thicknesses_dummy_1, final_pen_depth.n) * 1e12

data = {}
for idx, thickness in enumerate(thicknesses_um):
    if thickness not in data.keys():
        data[thickness] = ([], [])
    data[thickness][0].append(l_sheets_pH[idx])
    data[thickness][1].append(l_sheet_errs_pH[idx])

blues = get_blues(7, start=0.35, stop=0.95)
fit_color = get_blues(1, 1.0, 1.0)[0]

for thickness, (ls_pH, ls_err_pH) in data.items():

    if 0.0 < thickness < 0.025:
        color = blues[0]
    elif 0.025 < thickness < 0.06:
        color = blues[1]
    elif 0.06 < thickness < 0.15:
        color = blues[2]
    elif 0.15 < thickness < 0.3:
        color = blues[3]
    elif 0.3 < thickness < 0.5:
        color = blues[4]
    elif 0.5 < thickness < 1.2:
        color = blues[5]
    else:
        color = blues[6]

    ax.errorbar(
        np.repeat(thickness, len(ls_pH)),
        ls_pH,
        yerr=ls_err_pH,
        ls="",
        color=color,
        marker="o",
        label="data",
    )

pen_depth = emcee_result.params["pen_depth"].value * 1e6
pen_depth_err = emcee_result.params["pen_depth"].stderr * 1e6
pen_depth_str = r"$\mathrm{\lambda}$" + f" = {pen_depth:.2f} ± {pen_depth_err:.2f} μm"

ax.plot(
    thicknesses_dummy_1 * 1e6,
    l_sheets_fit_pH_1,
    color=fit_color,
    label="model",
)

l_sheet_saturated_pH = mu_0 * final_pen_depth.n * 1e12

ax.axhline(y=l_sheet_saturated_pH, color="k", linestyle='--', alpha=0.85)

ax.set_ylabel(r"$L_\mathrm{k/◻}$ (pH/$◻$)")
ax.set_xlabel(r"Film thickness (μm)")

ax.set_xscale("log")
ax.set_yscale("log")

ax.text(
    0.9,
    0.95,
    pen_depth_str,
    horizontalalignment="right",
    verticalalignment="top",
    transform=ax.transAxes,
)

ax.set_ylim(1, 350)

ax.set_xticks([0.01, 0.1, 1.0])
ax.get_xaxis().set_major_formatter(ticker.ScalarFormatter())

ax.set_yticks([1, 3, 10, 30, 100, 300])
ax.get_yaxis().set_major_formatter(ticker.ScalarFormatter())

#ax.legend(frameon=False, reverse=True)

fig.tight_layout()

figsavepath = CWD / "out/resonator_studies/penetration_depth.png"
plt.savefig(figsavepath, dpi=300, bbox_inches="tight")

plt.show()